In [116]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import re
from sklearn.preprocessing import MinMaxScaler
import time
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import OrderedDict
%matplotlib inline

In [117]:
dataframe = pd.read_csv('goodread_final_data.csv')

In [118]:
dataframe

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places
0,http://www.goodreads.com/book/show/136251.Harr...,Harry Potter and the Deathly Hallows,J.K. Rowling,71115,3093559,4.61,759.0,2007,True,"['Fantasy', 'Young Adult', 'Fiction']",['Locus Award Nominee for Best Young Adult Nov...,"['London, England', 'Hogwarts School of Witchc..."
1,http://www.goodreads.com/book/show/2767052-the...,The Hunger Games,Suzanne Collins,179564,6998314,4.32,284.0,2008,True,"['Young Adult', 'Fiction', 'Science Fiction']",['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Panem']"
2,http://www.goodreads.com/book/show/1.Harry_Pot...,Harry Potter and the Half-Blood Prince,J.K. Rowling,44399,2697998,4.57,652.0,2005,True,"['Fantasy', 'Young Adult', 'Fiction']",['Locus Award Nominee for Best Young Adult Nov...,"['Hogwarts School of Witchcraft and Wizardry',..."
3,http://www.goodreads.com/book/show/6.Harry_Pot...,Harry Potter and the Goblet of Fire,J.K. Rowling,52560,2888017,4.56,734.0,2000,True,"['Fantasy', 'Young Adult', 'Fiction']","['Hugo Award for Best Novel (2001)', "" Mythopo...","['Hogwarts School of Witchcraft and Wizardry',..."
4,http://www.goodreads.com/book/show/2.Harry_Pot...,Harry Potter and the Order of the Phoenix,J.K. Rowling,48162,2782996,4.50,870.0,2003,True,"['Fantasy', 'Young Adult', 'Fiction']",['Bram Stoker Award for Works for Young Reader...,"['Hogwarts School of Witchcraft and Wizardry',..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6394,http://www.goodreads.com/book/show/4820815-rac...,None,None,None,None,None,NaN,None,False,[],None,[]
6395,http://www.goodreads.com/book/show/32706309-ea...,East End Tales,Gilda O'Neill,22,93,3.51,50.0,2002,False,['Nonfiction'],NaN,[]
6396,http://www.goodreads.com/book/show/145342.How_...,How to Be a Girly Girl in Just Ten Days,Lisa Papademetriou,132,2948,4.05,163.0,2007,True,"['Childrens', 'Childrens', 'Middle Grade']",NaN,[]
6397,http://www.goodreads.com/book/show/470175.The_...,The Ransom of Mercy Carter,Caroline B. Cooney,180,1933,3.80,256.0,2000,False,"['Historical', 'Historical Fiction', 'Young Ad...",NaN,"['Canada', 'Deerfield, Massachusetts', 'Massac..."


In [119]:
dataframe['awards'] = dataframe.awards.fillna(0)

In [120]:
is_NaN = dataframe.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dataframe[row_has_NaN]
# dataframe = dataframe.dropna()

In [123]:
rows_with_NaN.to_csv('NaN-Values.csv', index=False) 

In [107]:
is_NaN = dataframe.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = dataframe[row_has_NaN]
# dataframe = dataframe.dropna()

In [108]:
rows_with_NaN

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places


In [109]:
dataframe["awards"] = dataframe.loc[dataframe['awards'] != 0, 'awards'].apply(eval).str.len()

C:\Users\Ellefson\miniconda3\envs\strive-ai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [110]:
dataframe

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places
0,http://www.goodreads.com/book/show/136251.Harr...,Harry Potter and the Deathly Hallows,J.K. Rowling,71115,3093559,4.61,759.0,2007,True,"['Fantasy', 'Young Adult', 'Fiction']",8.0,"['London, England', 'Hogwarts School of Witchc..."
1,http://www.goodreads.com/book/show/2767052-the...,The Hunger Games,Suzanne Collins,179564,6998314,4.32,284.0,2008,True,"['Young Adult', 'Fiction', 'Science Fiction']",41.0,"['District 12, Panem', 'Capitol, Panem', 'Panem']"
2,http://www.goodreads.com/book/show/1.Harry_Pot...,Harry Potter and the Half-Blood Prince,J.K. Rowling,44399,2697998,4.57,652.0,2005,True,"['Fantasy', 'Young Adult', 'Fiction']",3.0,"['Hogwarts School of Witchcraft and Wizardry',..."
3,http://www.goodreads.com/book/show/6.Harry_Pot...,Harry Potter and the Goblet of Fire,J.K. Rowling,52560,2888017,4.56,734.0,2000,True,"['Fantasy', 'Young Adult', 'Fiction']",9.0,"['Hogwarts School of Witchcraft and Wizardry',..."
4,http://www.goodreads.com/book/show/2.Harry_Pot...,Harry Potter and the Order of the Phoenix,J.K. Rowling,48162,2782996,4.50,870.0,2003,True,"['Fantasy', 'Young Adult', 'Fiction']",9.0,"['Hogwarts School of Witchcraft and Wizardry',..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6393,http://www.goodreads.com/book/show/6738284-the...,The Language of Life: DNA and the Revolution i...,Francis S. Collins,69,637,3.99,368.0,2009,False,"['Science', 'Nonfiction', 'Biology']",NaN,[]
6395,http://www.goodreads.com/book/show/32706309-ea...,East End Tales,Gilda O'Neill,22,93,3.51,50.0,2002,False,['Nonfiction'],NaN,[]
6396,http://www.goodreads.com/book/show/145342.How_...,How to Be a Girly Girl in Just Ten Days,Lisa Papademetriou,132,2948,4.05,163.0,2007,True,"['Childrens', 'Childrens', 'Middle Grade']",NaN,[]
6397,http://www.goodreads.com/book/show/470175.The_...,The Ransom of Mercy Carter,Caroline B. Cooney,180,1933,3.80,256.0,2000,False,"['Historical', 'Historical Fiction', 'Young Ad...",NaN,"['Canada', 'Deerfield, Massachusetts', 'Massac..."


In [111]:
dataframe['awards'] = dataframe.awards.fillna(0)

C:\Users\Ellefson\miniconda3\envs\strive-ai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [112]:
dataframe["awards"] = dataframe["awards"].astype(int)

C:\Users\Ellefson\miniconda3\envs\strive-ai\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [113]:
dataframe

,url,title,author,num_reviews,num_ratings,avg_rating,num_pages,original_publish_year,series,genres,awards,places
0,http://www.goodreads.com/book/show/136251.Harr...,Harry Potter and the Deathly Hallows,J.K. Rowling,71115,3093559,4.61,759.0,2007,True,"['Fantasy', 'Young Adult', 'Fiction']",8,"['London, England', 'Hogwarts School of Witchc..."
1,http://www.goodreads.com/book/show/2767052-the...,The Hunger Games,Suzanne Collins,179564,6998314,4.32,284.0,2008,True,"['Young Adult', 'Fiction', 'Science Fiction']",41,"['District 12, Panem', 'Capitol, Panem', 'Panem']"
2,http://www.goodreads.com/book/show/1.Harry_Pot...,Harry Potter and the Half-Blood Prince,J.K. Rowling,44399,2697998,4.57,652.0,2005,True,"['Fantasy', 'Young Adult', 'Fiction']",3,"['Hogwarts School of Witchcraft and Wizardry',..."
3,http://www.goodreads.com/book/show/6.Harry_Pot...,Harry Potter and the Goblet of Fire,J.K. Rowling,52560,2888017,4.56,734.0,2000,True,"['Fantasy', 'Young Adult', 'Fiction']",9,"['Hogwarts School of Witchcraft and Wizardry',..."
4,http://www.goodreads.com/book/show/2.Harry_Pot...,Harry Potter and the Order of the Phoenix,J.K. Rowling,48162,2782996,4.50,870.0,2003,True,"['Fantasy', 'Young Adult', 'Fiction']",9,"['Hogwarts School of Witchcraft and Wizardry',..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6393,http://www.goodreads.com/book/show/6738284-the...,The Language of Life: DNA and the Revolution i...,Francis S. Collins,69,637,3.99,368.0,2009,False,"['Science', 'Nonfiction', 'Biology']",0,[]
6395,http://www.goodreads.com/book/show/32706309-ea...,East End Tales,Gilda O'Neill,22,93,3.51,50.0,2002,False,['Nonfiction'],0,[]
6396,http://www.goodreads.com/book/show/145342.How_...,How to Be a Girly Girl in Just Ten Days,Lisa Papademetriou,132,2948,4.05,163.0,2007,True,"['Childrens', 'Childrens', 'Middle Grade']",0,[]
6397,http://www.goodreads.com/book/show/470175.The_...,The Ransom of Mercy Carter,Caroline B. Cooney,180,1933,3.80,256.0,2000,False,"['Historical', 'Historical Fiction', 'Young Ad...",0,"['Canada', 'Deerfield, Massachusetts', 'Massac..."


In [114]:
dataframe.to_csv('goodread_final_5596.csv', index=False)